In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
import math
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import tree
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression


In [2]:
train = pd.read_csv("train.csv").set_index('PassengerId')
test = pd.read_csv("test.csv").set_index('PassengerId')

## EDA

In [3]:
test.head(), test.columns

(             Pclass                                          Name     Sex  \
 PassengerId                                                                 
 892               3                              Kelly, Mr. James    male   
 893               3              Wilkes, Mrs. James (Ellen Needs)  female   
 894               2                     Myles, Mr. Thomas Francis    male   
 895               3                              Wirz, Mr. Albert    male   
 896               3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   
 
               Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
 PassengerId                                                       
 892          34.5      0      0   330911   7.8292   NaN        Q  
 893          47.0      1      0   363272   7.0000   NaN        S  
 894          62.0      0      0   240276   9.6875   NaN        Q  
 895          27.0      0      0   315154   8.6625   NaN        S  
 896          22.0      1      1  3101298  1

In [4]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
y = train['Survived']
X = train.drop(['Survived'], axis = 'columns')

In [7]:
X.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


In [9]:
X.isnull().sum().sort_values(ascending=False)

Cabin       687
Age         177
Embarked      2
Pclass        0
Name          0
Sex           0
SibSp         0
Parch         0
Ticket        0
Fare          0
dtype: int64

### too much null in Cabin, name and ticket looks like not very important just like id

In [10]:
X = X.drop(['Cabin','Name','Ticket'], axis = 'columns')

In [11]:
X['Age'].fillna(X['Age'].mean(), inplace = True)

In [12]:
X.isnull().sum().sort_values(ascending=False)

Embarked    2
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [13]:
test.isnull().sum().sort_values(ascending=False)

Cabin       327
Age          86
Fare          1
Pclass        0
Name          0
Sex           0
SibSp         0
Parch         0
Ticket        0
Embarked      0
dtype: int64

In [14]:
test = test.drop(['Cabin','Name','Ticket'], axis = 'columns')

In [15]:
test['Age'].fillna(X['Age'].mean(), inplace = True)

In [16]:
test.isnull().sum().sort_values(ascending=False)

Fare        1
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [17]:
X['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [18]:
X['Embarked'].fillna(X['Embarked'].mode()[0], inplace=True)

In [19]:
X.isnull().sum().sort_values(ascending=False)

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [20]:
test['Fare'].fillna(X['Fare'].mean(), inplace = True)

In [21]:
test.isnull().sum().sort_values(ascending=False)

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [22]:
X = pd.get_dummies(X,columns=['Sex', 'Embarked'])


In [23]:
X.columns, len(X.columns)

(Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
        'Embarked_C', 'Embarked_Q', 'Embarked_S'],
       dtype='object'),
 10)

In [24]:
test.columns, len(test.columns)

(Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object'),
 7)

In [25]:
test = pd.get_dummies(test)

In [26]:
test.columns, len(test.columns)

(Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
        'Embarked_C', 'Embarked_Q', 'Embarked_S'],
       dtype='object'),
 10)

### pclass is categorical!!!

In [28]:
X['Pclass'].unique()

array([3, 1, 2])

In [29]:
X['Pclass'] =X['Pclass'].astype('category')

In [30]:
X = pd.get_dummies(X,columns=['Pclass'])

In [31]:
test['Pclass'] = test['Pclass'].astype('category')

In [32]:
test = pd.get_dummies(test,columns=['Pclass'])

In [33]:
len(test.columns) , len(X.columns)

(12, 12)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
lr = LogisticRegression()

In [39]:
lr.fit(X_train,y_train)

/home/victoria/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [41]:
y_pred = lr.predict(X_test)

In [42]:
accuracy_score(y_pred,y_test)

0.8059701492537313

In [43]:
predictions = lr.predict(test)

In [46]:
output = pd.DataFrame({'PassengerId': test.index, 'Survived': predictions})
output.to_csv('submissionLR.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [47]:
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')

X_test.to_pickle('X_test.pkl')
y_test.to_pickle('y_test.pkl')


In [48]:
test.to_pickle('test.pkl')
